In [1]:
#https://qiita.com/kenchin110100/items/b34f5106d5a211f4c004

In [2]:
import json
import glob
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
import MeCab
import collections
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import optimizers
from chainer import optimizer
from chainer import serializers

/Users/admin/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
os.makedirs('temp_data',exist_ok=True)

json_list = glob.glob('./data/projectnextnlp-chat-dialogue-corpus/json/rest1046/*')

In [4]:
data_all = []
for json_name in json_list:
    data = []
    with open(json_name,'r') as f:
        j = json.load(f)
    temp = j['turns']
    for k in range(len(temp)):
        data.append((temp[k]['utterance']))
    data_all.append(data)
with open('./temp_data/data_all.pkl','wb') as f:
    pickle.dump(data_all,f)
print(data_all[0])
print(data_all[1])

['おはようございます。熱中症に気をつけて。', 'ありがとうございます。今日も暑いですね。', '熱中症に気をつけないんですか？', 'もちろん気をつけますよ', '熱中症に気をつけか？？', '気をつけますよ', 'ありがと', '気をつけてくださいね', 'はーい', '暑い日もあれば、雨がたくさん降る日もありますね。', '雨は大丈夫ですね', 'そうなんですか？', 'そうだよ', '何が大丈夫なんですか？', '雨は心配ないらしい', 'なぜですか？', 'どした', '雨が心配ないというのはなぜですか？', '雨は心配してますよ', 'さっき言ってたことと違いますよ', '雨は大丈夫だそうです']
['こんにちは。声かけてくれるのを待ってたんだ。', 'こんにちは。今日は風があって助かりますね。', '風は欲しいですね', '真夏は風はいいですね。', '気持ちがいいですね', 'はい。プールとか海にも行きたいです。', '風は心地よいですね', '風がないとジリジリ暑いです。', '風が心地良いですよねえ', 'はい。髪は乱れますが。', '風が心地良いですね', '冬は風があると寒いです。', '風は爽やかですね', '爽やかなのが一番です。', '風は清々しいですね', '洗濯物も良く乾きます。', '風は穏やかですね', 'そよ風がいいですね。', 'そよ風が心地よいですね', '早く涼しくなるといいです。', 'そよ風はいいですね']


In [5]:
m = MeCab.Tagger ("-Owakati")

data_all_wakati = []
for data in data_all:
    data_wakati = []
    for d in data:
        d_wakati = m.parse(d).split()
        data_wakati.append(d_wakati)
    data_all_wakati.append(data_wakati)
with open('./temp_data/data_all_wakati.pkl','wb') as f:
    pickle.dump(data_all_wakati,f)
print(data_all_wakati[0])
print(data_all_wakati[1])

[['おはよう', 'ござい', 'ます', '。', '熱中', '症', 'に', '気', 'を', 'つけ', 'て', '。'], ['ありがとう', 'ござい', 'ます', '。', '今日', 'も', '暑い', 'です', 'ね', '。'], ['熱中', '症', 'に', '気', 'を', 'つけ', 'ない', 'ん', 'です', 'か', '？'], ['もちろん', '気', 'を', 'つけ', 'ます', 'よ'], ['熱中', '症', 'に', '気', 'を', 'つけ', 'か', '？', '？'], ['気', 'を', 'つけ', 'ます', 'よ'], ['ありがと'], ['気', 'を', 'つけ', 'て', 'ください', 'ね'], ['はーい'], ['暑い', '日', 'も', 'あれ', 'ば', '、', '雨', 'が', 'たくさん', '降る', '日', 'も', 'あり', 'ます', 'ね', '。'], ['雨', 'は', '大丈夫', 'です', 'ね'], ['そう', 'な', 'ん', 'です', 'か', '？'], ['そう', 'だ', 'よ'], ['何', 'が', '大丈夫', 'な', 'ん', 'です', 'か', '？'], ['雨', 'は', '心配', 'ない', 'らしい'], ['なぜ', 'です', 'か', '？'], ['どし', 'た'], ['雨', 'が', '心配', 'ない', 'という', 'の', 'は', 'なぜ', 'です', 'か', '？'], ['雨', 'は', '心配', 'し', 'て', 'ます', 'よ'], ['さっき', '言っ', 'て', 'た', 'こと', 'と', '違い', 'ます', 'よ'], ['雨', 'は', '大丈夫', 'だ', 'そう', 'です']]
[['こんにちは', '。', '声', 'かけ', 'て', 'くれる', 'の', 'を', '待っ', 'て', 'たん', 'だ', '。'], ['こんにちは', '。', '今日', 'は', '風', 'が', 'あっ', 'て', '助かり', 'ます', 'ね', '。'], ['風', 'は', '

In [6]:
#############
# for saving comutational resource, I reduce samples used
data_all_wakati = data_all_wakati
#############

In [7]:
#https://qiita.com/Kodaira_/items/172c73b68cd15ad1ead6
word2id = collections.defaultdict(lambda: len(word2id) )
    
all_words = []
for data_wakati in data_all_wakati:
    for d_wakati in data_wakati:
        all_words += d_wakati
null_id = word2id['NULL']
end_id  = word2id['END']
ids = [word2id[word] for word in all_words]
word2id_dict = dict(word2id)
id2word_dict = {v:k for k, v in word2id_dict.items()}
n_words = len(word2id_dict)

with open('./temp_data/word2id_dict.pkl','wb') as f:
    pickle.dump(word2id_dict,f,protocol=2)
with open('./temp_data/id2word_dict.pkl','wb') as f:
    pickle.dump(id2word_dict,f,protocol=2)

print('word number is {}'.format(n_words))
print(word2id_dict['おはよう'])
print(id2word_dict[2])

word number is 6923
2
おはよう


In [8]:
def convert(sentence, dictionary):
    return [dictionary[word] if word in dictionary.keys() else -1 for word in sentence]
def process(ids):
    return [end_id] + ids + [null_id] * (th_seq_length - len(ids) - 1)
questions = []
answers   = []
th_seq_length = 20

for data_wakati in data_all_wakati:
    for k in range(len(data_wakati)-1):
        q = convert(data_wakati[k], word2id_dict)
        a = convert(data_wakati[k+1], word2id_dict)
        if max(len(q),len(a)) >= th_seq_length:
            continue
        else:
            questions.append(process(q))
            answers.append(process(a))
print(questions[0])
print(answers[0])

questions = np.array(questions)
answers   = np.array(answers)
with open('./temp_data/questions.pkl','wb') as f:
    pickle.dump(questions,f,protocol=2)
with open('./temp_data/answers.pkl','wb') as f:
    pickle.dump(answers,f,protocol=2)

n_seq = len(questions)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 5, 0, 0, 0, 0, 0, 0, 0]
[1, 13, 3, 4, 5, 14, 15, 16, 17, 18, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
class RNN(chainer.Chain):
    def __init__(self, n_words, n_hiddens):
        super().__init__()
        with self.init_scope():
            self.embed = L.EmbedID(n_words, n_hiddens)
            self.lstm = L.LSTM(n_hiddens, n_hiddens)
            self.fc = L.Linear(n_hiddens, n_words)
            
    def reset_state(self):
        self.lstm.reset_state()

    def get_state(self):
        return self.lstm.c, self.lstm.h
    
    def set_state(self, c, h):
        self.lstm.c = c
        self.lstm.h = h
        
    def __call__(self,x):
        h = self.embed(x)
        h = self.lstm(h)
        h = self.fc(h)
        return h
    

def batch_sampling(x,y,bs):
    r = np.random.permutation(n_seq)[:bs]
    x_batch = x[r]
    y_batch = y[r]
    return x_batch, y_batch

In [10]:
enc = RNN(n_words, 300)
dec = RNN(n_words, 300)

In [95]:
serializers.load_npz('./temp_data/enc0',enc)
serializers.load_npz('./temp_data/dec0',dec)

In [96]:
bs = 32
ite_pre = 100
ite_seq2seq = 30000

In [97]:
opt_enc = optimizers.Adam()
opt_enc.setup(enc)
opt_enc.add_hook(optimizer.GradientClipping(1))
opt_dec = optimizers.Adam()
opt_dec.setup(dec)
opt_dec.add_hook(optimizer.GradientClipping(1))

In [43]:
loss_rec = []
for _ in range(ite_pre):
    loss = 0
    enc.reset_state()
    q_batch, _ = batch_sampling(questions, answers, bs)
    q_batch = q_batch[:,::-1]
    for t in range(th_seq_length - 1):
        p = enc(q_batch[:,t])
        loss += F.softmax_cross_entropy(p, q_batch[:,t+1])
    enc.cleargrads()
    loss.backward()
    opt_enc.update()
    print(loss.data)
    loss_rec.append(loss.data)
serializers.save_npz('temp_data/pre_enc',enc)
plt.plot(loss_rec)

50.67815


KeyboardInterrupt: 

In [113]:
loss_rec = []
end_ids = np.array([end_id] * bs)
for ite in range(ite_seq2seq):
    if ite%100 == 0:
        serializers.save_npz('temp_data/enc{}'.format(ite),enc)
        serializers.save_npz('temp_data/dec{}'.format(ite),dec)
    loss = 0
    enc.reset_state()
    dec.reset_state()
    q_batch, a_batch = batch_sampling(questions, answers, bs)
    q_batch = q_batch[:,::-1]
    for t in range(th_seq_length - 1):
        p = enc(q_batch[:,t])

    c,h = enc.get_state()
    dec.set_state(c,h)

    id_preds = []
    for t in range(1, th_seq_length):
        if t == 1:
            p = dec(np.array(end_ids))
        else:
            p = dec(np.array(pred_id))

        loss += F.softmax_cross_entropy(p, a_batch[:,t])
        pred_id = F.argmax(p,axis=1).data
        id_preds.append(pred_id)
    enc.cleargrads()
    dec.cleargrads()
    loss.backward()
    opt_enc.update()
    opt_dec.update()

    print('----')
    print(loss.data)
    print(' '.join(convert(a_batch[0,1:],id2word_dict)))
    print(' '.join(convert(np.vstack(id_preds)[:,0],id2word_dict)))
    loss_rec.append(loss.data)

----
41.440414
神奈川 県 代表 に なれ て ない ん です よ ねぇ NULL NULL NULL NULL NULL NULL NULL NULL
そう は は です です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
41.123554
食事 は 一緒 に し たい です か ？ ？ NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
38.74594
メロン も 好き です 。 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
40.8411
や まて せ ん で は あり ませ ん NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は は です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
38.484573
気持ち が いい らしい です よ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
33.67592
です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL NULL N

----
29.73927
メロン は 大好き です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
42.566418
神 を 忘れ ます NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
40.797924
悟り を 開い てる の ？ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
35.24072
何 か 食べ まし た か ？ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
41.16225
トランプ とか し たい なぁ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
44.071102
それ じゃ 、 芝生 の 上 に 、 レジャー シート 敷い て 、 座り ましょ う 。 NULL NULL
そう 

----
33.471302
海 は 一 人 で 行っ た なぁ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
海 は です です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
45.83536
臨場 感 は すごい です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
スイカ は です です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
36.894028
たのしい です よ ね 。 でも クーラー の ある ところ に い たい です 。 NULL NULL NULL NULL
スイカ は です です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
38.19767
そうですね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
スイカ は です です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
35.97926
紫外線 は お 肌 の 大敵 だ から ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
スイカ は です です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
43.415627
ピリ 辛 な ところ が アクセント に なっ て 良い です NULL NULL NULL NULL NULL NULL NULL NULL
スイカ は です です NULL NULL N

----
38.10069
プール は いい そう です よ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
スイカ は は です です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
40.673443
私 はいり ませ ん が 、 子供 は 喜ん で い ます 。 NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
41.307102
洋服 は 可愛らしい です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
スイカ は が です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
39.078773
かき氷 は 食べ まし た か ？ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
46.938473
この あたり の 好み は 似 て い ます ね NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
44.229393
大事 です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は N

----
38.903084
雨 は 心配 ない です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
海 は が です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
40.317318
支障 が 仕事 に 出る かも しれ ない NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
海 は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
38.279186
オーク ション は 楽しい です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
海 は が です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
43.05983
うい NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
うん NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
40.82296
熱中 症 は 大丈夫 です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
海 は は です です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
39.051865
競技 は 奥 が 深い です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NUL

...

----
35.896038
スイカ に かぶりつき ます NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
退屈 は は です です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
39.40837
海 へ 泳ぎ に 行き ます か ？ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
何 NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
46.219448
毎年 泳い で ます か ？ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
何 は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
40.55155
スイカ は 食べる の が おいしい です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
スイカ は いい です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
37.7583
作家 さん です か NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は は です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
40.521152
ありがとう ござい ます NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL N

----
34.705334
おーい NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
42.17061
前向き です ね 。 すら れ た ん です か ？ NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
36.091297
ナミ に かぶせ ます よ ねー NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
38.593647
朝 まで つけ っぱなし に し てる と 体 が だるく なっ ちゃ い ます ね NULL NULL NULL
そう は NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
48.884815
我が家 は 大好き です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
39.148376
旅行 は 好き です か ？ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NU

----
43.454636
私 は キーボード に 入力 し て い ます 。 NULL NULL NULL NULL NULL NULL NULL NULL NULL
退屈 は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
37.162163
予防 が 大切 です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
スイカ は は です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
33.44116
食べ ます NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そうですね は は です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
41.076866
お 天気 を 祈り ます NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
36.761036
もう そろそろ 透析 導入 に なる ん じゃ ない の ？ NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
36.10187
買い物 は 女 同士 が 楽しい です ね NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NUL

----
41.29874
小鉢 の 酢の物 が えらく 美味しい わ NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
40.019882
はい 、 肌寒い です 。 保育園 の プール も でき ない でしょ う ね 。 NULL NULL NULL NULL
海 は は です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
35.96798
まだ 浅い 話 だ と 思う よ 。 都道府県 レベル 。 NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
38.313274
ビール は 買わ なく て いい です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL です NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
32.511417
鰹節 だけ は 嫌 かも NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
そう は NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
----
40.250854
朝 から 海 に 行き ます NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL NULL
海 は に です NULL

KeyboardInterrupt: 

In [14]:
plt.plot(loss_rec)

NameError: name 'loss_rec' is not defined

In [102]:
def chat(inp, argmax_decoding = True):
    inp_wakati = m.parse(inp).split()
    inp_id = convert(inp_wakati, word2id_dict)
    print(inp_id)
    q = np.array(process(inp_id))

    if max(np.mean(q == questions,axis=1)) == 1:
        print('training data include this question')
        
    q = q[::-1]
    
    id_preds = []

    enc.reset_state()
    dec.reset_state()
    for t in range(th_seq_length - 1):
        p = enc(q[t:t+1])

    c,h = enc.get_state()
    dec.set_state(c,h)
    ps = []
    for t in range(1, th_seq_length):
        if t == 1:
            p = dec(np.array([end_id]))
        else:
            p = dec(np.array([pred_id]))
            
        p = F.softmax(p).data[0]
        
        if argmax_decoding == True:
            pred_id = np.argmax(p)
        else:
            pred_id = np.random.choice(range(n_words), p = p)
        p = p[pred_id]
        ps.append(p)
        id_preds.append(int(pred_id))
        if pred_id == null_id:
            break
#     p_joint = 1
#     for p in ps:
#         p_joint *= p
#     print(p_joint ** (1/len(ps)))

    print(' '.join(convert(id_preds,id2word_dict)))

In [104]:
inp = '暑いですね'
chat(inp)

[16, 17, 18]
training data include this question
暑い です NULL


In [106]:
inp = '海に行こう'
chat(inp)

[72, 8, 682, 191]
どこ 水着 が に NULL


In [108]:
inp = 'どうしたの？'
chat(inp)

[408, 50, 47, 49, 22]
training data include this question
なに ー NULL


In [110]:
inp = 'ダメですね。もうちょっと頑張ってください'
chat(inp)

[769, 17, 18, 5, 480, 151, 1522, 12, 26]
果物 は は です ね NULL


In [112]:
inp = '無理っぽい？'
chat(inp)

[1069, 1901, 22]
スイカ を で 勉強 し て ない です NULL


In [113]:
inp = 'あなたの仕事はなんですか？'
chat(inp)

[108, 49, 272, 37, 223, 17, 21, 22]
研究 です NULL


In [114]:
inp = 'こんにちは'
chat(inp)

[56]
training data include this question
こんにちは NULL


In [115]:
inp = 'おはようございます'
chat(inp)

[2, 3, 4]
training data include this question
おはよう NULL


In [116]:
inp = 'あまり調子が良くない'
chat(inp)

[348, 557, 33, 90, 19]
ブログ は 、 何 NULL


In [117]:
inp = 'もうちょっと心配してください'
chat(inp)

[480, 151, 43, 50, 12, 26]
予防 が 大切 です ね NULL


In [118]:
inp = '海に遊びに行こうと思います'
chat(inp)

[72, 8, 519, 8, 682, 191, 54, 334, 4]
海 に 行っ て ない ？ NULL


In [120]:
inp = '海に遊びに行こうと思いますが、一緒にどうですか？'
chat(inp)

[72, 8, 519, 8, 682, 191, 54, 334, 4, 33, 31, 392, 8, 408, 17, 21, 22]
左手 に 行き を です NULL


In [122]:
inp = '予防注射は大切ですね'
chat(inp)

[220, 4178, 37, 221, 17, 18]
確か に じゃ です ね NULL


In [123]:
inp = '一緒に行こう'
chat(inp)

[392, 8, 682, 191]
行こ う NULL


In [124]:
inp = 'お茶ほしい'
chat(inp)

[869, 520]
へー が いい 悪い です ね NULL


In [125]:
inp = '酵素高い'
chat(inp)

[725, 149]
酵素 は する です NULL


In [127]:
inp = 'さようなら'
chat(inp)

[588]
training data include this question
また 会う た まで NULL


In [128]:
inp = 'ばいばい'
chat(inp)

[587, 587]
training data include this question
ばい ば あり NULL


In [130]:
inp = '東京に遊びに行きましょう'
chat(inp)

[2191, 8, 519, 8, 73, 241, 191]
ねえ は 必要 です NULL


In [131]:
inp = '世界が平和でありますように'
chat(inp)

[2012, 33, 2371, 139, 36, 4, 540, 8]
クーラー で は です か ね ？ NULL


In [132]:
inp = '今日は何をして遊びますか？'
chat(inp)

[14, 37, 42, 10, 50, 12, 519, 4, 21, 22]
退屈 を 食べ たり し て NULL


In [133]:
inp = '何の話ですか？'
chat(inp)

[42, 49, 341, 17, 21, 22]
training data include this question
需要 は 全く ない です NULL


In [134]:
inp = '需要がないのは良くない'
chat(inp)

[3632, 33, 19, 49, 37, 90, 19]
需要 は 無理 ん だ ． NULL


In [148]:
inp = '熱中症になれよ'
chat(inp)

[6, 7, 8, 1836, 24]
熱中 は 暑い に NULL


In [153]:
inp = 'ブラックな素晴らしきお仕事'
chat(inp)

[310, 40, 323, 189, 272]
なかっ が 好き です ね NULL
